# Lecture 4 : Functions and Pointers

# Part 1 : Computing $\displaystyle\binom{n}{k}$

## Consider the problem of calculating $$\displaystyle\binom{n}{k} = \displaystyle\frac{n!}{(n-k)!k!}$$

## Due to overflow we cannot compute $n!$ for even modest $n$ using a *long long int*.  

## For example $256!$ has 507 digits!

## Fortunately we can compute $\binom{n}{k}$ starting with $\binom{n}{0} = 1$ iteratively by making use of the formula:

$$\displaystyle\binom{n}{i+1} = \displaystyle\binom{n}{i} \displaystyle\frac{n-i}{i+1}$$

## The formula can be shown directly from the definition of $\binom{n}{k}$:

$$\displaystyle\binom{n}{i} \displaystyle\frac{n-i}{i+1} = \displaystyle\frac{n!}{(n-i)!i!} \, \displaystyle\frac{n-i}{i+1} = \displaystyle\frac{n!}{(n-i-1)! (i+1)!} =
\displaystyle\frac{n!}{(n-(i+1))! (i+1)!} =
\displaystyle\binom{n}{i+1}$$

## Let's illustrate the use of the formula by calculating $\binom{256}{4}$ without calculating $256!$

## We first note that $\binom{256}{0} = 1$.  To compute $\binom{256}{1}$ we use the formula with $i=0$ and note that
$$\displaystyle\binom{256}{1} = \displaystyle\binom{256}{0} \displaystyle\frac{256-0}{0+1} = 256$$

## To compute $\binom{256}{2}$ we use the formula with $i=1$ and note that
$$\displaystyle\binom{256}{2} = \displaystyle\binom{256}{1} \displaystyle\frac{256-1}{1+1} = 256*255/2 = 32640$$

## It is critical that we first calculate $256*255$ before dividing by $2$ since $255/2$ is not an integer!

## To compute $\binom{256}{3}$ we use the formula with $i=2$ and note that
$$\displaystyle\binom{256}{3} = \displaystyle\binom{256}{2} \displaystyle\frac{256-2}{2+1} = 32640*254/3 = 2763520$$

## To compute $\binom{256}{4}$ we use the formula with $i=3$ and note that
$$\displaystyle\binom{256}{4} = \displaystyle\binom{256}{3} \displaystyle\frac{256-3}{3+1} = 2763520*253/4 = 174792640$$

## We can use a for loop in C to automate this iterative process:

In [1]:
%%writefile binom_v1.c
#include <stdio.h>
#include <stdlib.h>

int main (int argc, char* argv[]) {
    if (argc < 3) {
        printf ("command usage: %s %s %s\n",argv[0],"n","k");
        return 1;
    }
    int n = atoi(argv[1]);
    int k = atoi(argv[2]);
    long long answer = 1;
    for (int i=0;i<=k-1;i++) {
        answer = answer*(n-i)/(i+1);
    }
    printf ("%lld\n",answer);
}

Overwriting binom_v1.c


In [2]:
!gcc -o binom_v1 binom_v1.c

In [3]:
!./binom_v1 256 4

174792640


In [4]:
!./binom_v1 256 8

409663695276000


## As our programs become more complicated we will want to make use of functions to make our code more readable and easier to maintain.

## Here is a version where we calculate $\binom{n}{k}$ using a function.  

## Note that the syntax for C functions is similar to Java functions.

In [5]:
%%writefile binom_v2.c
#include <stdio.h>
#include <stdlib.h>

long long binom(int n, int k) {
    long long answer = 1;
    for (int i=0;i<=k-1;i++) {
        answer = answer*(n-i)/(i+1);
    }
    return answer;
}

int main (int argc, char* argv[]) {
    if (argc < 3) {
        printf ("command usage: %s %s %s\n",argv[0],"n","k");
        return 1;
    }
    int n = atoi(argv[1]);
    int k = atoi(argv[2]);
    long long answer = binom(n,k);
    printf ("%lld\n",answer);
}

Overwriting binom_v2.c


In [6]:
!gcc -o binom_v2 binom_v2.c

In [7]:
!./binom_v2 256 8

409663695276000


## Note that $\binom{256}{16} = 10078751602022313874633200$ is too large to fit into a C *long long int*.

In [8]:
!./binom_v2 256 16

107333149995938127


## As expected our program does not compute the correct answer.  

## It would be far preferable for our program to output an error rather than the incorrect answer.

## This program shows that C does not automatically check for overflow.   

## Note that Java also does not automatically check for overflow.  

## Thus we need to add such error checking code ourselves.  

## In our *binom_v2* program a possible overflow occurs when calculating **answer*(n-i)**.  

## We can detect a multiplication overflow be utilizing the following result regarding integer multiplication and division.  

## Let a, b, c be integers and / denote integer division.  Then we have
$$ab > c \text{ if and only if } a > c/b$$

## In C the largest possible long long int is denoted LLONG_MAX (a constant defined in limits.h).  

## Thus using the result we can check if $ab > \text{LLONG_MAX}$ (i.e. overflow) by testing to see if
$$a > \frac{\text{LLONG_MAX}}{b}$$

## Here is a version of our *binom* program that detects overflow.

In [9]:
%%writefile binom_v3.c
#include <stdio.h>
#include <stdlib.h>
#include <limits.h>

long long binom(int n, int k) {
    long long answer = 1;
    for (int i=0;i<=k-1;i++) {
        if (answer > LLONG_MAX/(n-i)) {
            printf ("overflow detected calculating %d choose %d\n",n,k);
            exit(1);
        }
        answer = answer*(n-i)/(i+1);
    }
    return answer;
}

int main (int argc, char* argv[]) {
    if (argc < 3) {
        printf ("command usage: %s %s %s\n",argv[0],"n","k");
        return 1;
    }
    int n = atoi(argv[1]);
    int k = atoi(argv[2]);
    long long answer = binom(n,k);
    printf ("%lld\n",answer);
}

Overwriting binom_v3.c


In [10]:
!gcc -o binom_v3 binom_v3.c

In [11]:
!./binom_v3 256 16

overflow detected calculating 256 choose 16


## In this version of the program our *binom* function calls exit(1) which abnormally terminates the program if an overflow is encountered.

## We may prefer to handle such errors without terminating the program.  

## For example, we could instead have our function *binom* return a flag indicating success or failure of the computation.

## In this case we want to have our function return multiple values (a flag indicating success/failure as well as the value of the $\binom{n}{k}$ calculation).  

## There are multiple ways to write functions in C that return more than one value.

## One common way is to allow the function to modify one or more of its arguments using **C pointers**.

# Part 2 : Introduction to Pointers in C

## C pointers are memory addresses and can be used to assign values to variables **indirectly**.

## Pointers are not available in Java.  This is one of the major differences between C and Java.

## Consider the following example:

In [12]:
%%writefile pointer.c
#include <stdio.h>

int main () {
    int a = 3;
    int* p = &a;
    *p = 4;
    printf ("a = %d and p = %p\n",a,p);
}

Overwriting pointer.c


## Notes:

* ### There are two new concepts in line 6.  First note that the type of the variable *p* is *int**.  In other words, $p$ is a **pointer** to an integer.  

* ### The second new idea in line 6 is that & is a C operator that takes the **address** of a variable.  In particular, &a is the address (or location) where the variable *a* is stored in memory.

* ### Putting these two ideas together, in line 6 we **declare** the variable *p* to be a pointer to an integer and we **initialize** the variable *p* to the memory address of *a*.  

* ### In other words, line 6 initializes the integer pointer *p* to **point** to the integer variable *a*.

* ### In line 7, the * is the C operator that **dereferences** a pointer.  In other words, **p* is an alias for the variable that the pointer *p* points to which is the variable *a*.  Thus, line 7 assigns the value 4 to the variable *a*.  

* ### In line 8, the *%p* is the format specifier for pointer.  Pointers in C are printed in hexadecimal (see output below).

In [13]:
!gcc -o pointer pointer.c

In [14]:
!./pointer

a = 4 and p = 0x7ffd4257955c


## Note that we changed the value of $a$ to $4$ indirectly using the pointer $p$ and that the memory address contained in the pointer variable *p* is printed in hexadecimal.  

## At this point you may be wondering why someone would want to use C pointers.  After all line 7 could be replaced with $a=4$.  This change would not effect the output of the program and the code would certainly be easier to read and understand.  

## The benefits (and dangers) of using C pointers will become more clear as we progress.  

## To understand our first application of C pointers we need to learn about **C function call semantics**.


# Part 3 : C Function call Semantics

## Consider the following C code that attempts to add 2 integers.

## Note that we are attempting to return the answer through the function argument *c* instead of a return value (we may want to use the return value for something else such as an overflow flag).

In [15]:
%%writefile sum2_v1.c
#include <stdio.h>

void add(int a, int b, int c) {
    c = a + b;
}

int main () {
    int x = 3;
    int y = 7;
    int sum = 0;
    add(x,y,sum);
    printf ("sum = %d\n",sum);
}

Overwriting sum2_v1.c


In [16]:
!gcc -o sum2_v1 sum2_v1.c

In [17]:
!./sum2_v1

sum = 0


## The code outputs 0 which is the initial value of sum assigned in line 12 instead of the sum of 3 and 7 which is 10.   

## The call to the function *add2* did not change the variable *sum*.

## **C passes arguments to functions by value (also called pass by copy).**

## When the function *add* is called, the values of x, y, and sum are copied to the variables a, b, and c.  

## In particular, since line 5 operates on copies of the arguments, the value of the variable sum is not changed by the function.

## In C we can work around the pass by value semantics by passing a pointer to the sum variable instead of the value of the sum variable.  Using a pointer to sum, the add2 function will be able to change the value of sum indirectly.  

## Here is the new version that gives the expected output:

In [18]:
%%writefile sum2_v2.c
#include <stdio.h>

void add(int a, int b, int* c) {
    *c = a + b;
}

int main () {
    int x = 3;
    int y = 7;
    int sum = 0;
    add(x,y,&sum);
    printf ("sum = %d\n",sum);
}

Overwriting sum2_v2.c


## Notes:

* ### In line 4 we declare the third argument c to be an integer pointer.

* ### In line 5 we dereference the pointer c using the syntax **c* so that the variable that the pointer c points to (which is the variable sum in this example) is assigned the value of *a+b*.

* ### In line 12 we pass a pointer to the sum as our third argument using the *&sum* syntax (recall that & is the address operator in C).

In [19]:
!gcc -o sum2_v2 sum2_v2.c

In [20]:
!./sum2_v2

sum = 10


# Part 4 : Computing $\displaystyle\binom{n}{k}$ Revisited

## Here is the final version of our *binom* program that checks for overflow and uses a pointer to return the answer.

In [21]:
%%writefile binom_v4.c
#include <stdio.h>
#include <stdlib.h>
#include <limits.h>

// returns 0 if success
// returns 1 if overflow
int binom(int n, int k, long long* answer) {
    *answer = 1;
    for (int i=0;i<=k-1;i++) {
        if (*answer > LLONG_MAX/(n-i)) {
            return 1;
        }
        *answer = *answer*(n-i)/(i+1);
    }
    return 0;
}

int main (int argc, char* argv[]) {
    if (argc < 3) {
        printf ("command usage: %s %s %s\n",argv[0],"n","k");
        return 1;
    }
    int n = atoi(argv[1]);
    int k = atoi(argv[2]);
    long long answer;
    if (binom(n,k,&answer)) {
        printf ("overflow detected calculating %d choose %d\n",n,k);
    } else {
        printf ("%lld\n",answer);
    }
}

Overwriting binom_v4.c


## Notes:

* ### C does not have a Boolean datatype.  Instead we typically use integers for Booleans.  In C, 0 is FALSE and any other integer is TRUE.  Thus, on line 27 we are checking to see if binom returned any value other than 0.  

In [22]:
!gcc -o binom_v4 binom_v4.c

In [23]:
!./binom_v4 256 8

409663695276000


In [24]:
!./binom_v4 256 16

overflow detected calculating 256 choose 16


# Part 5 : Reading Input from *stdin* Using *scanf*

## In the previous lecture we learned how to use command line arguments to get input from the user at runtime.  

## Command line arguments only make sense for a small amount of input.  

## For larger inputs (such as a file containing data) we can read the input from *stdin* using *scanf* (formatted scan).  

## Arguments to *scanf* are passed by pointer so that they can be modified (indirectly) to contain the next inputs from *stdin*.  

## The return value of *scanf* gives the number of fields that were successfully converted and assigned to variables.

## One key use of the return value of *scanf* is to check for *end of file*.

## Here is a C program that sums the numbers read from *stdin*.  

In [25]:
%%writefile sum.c
#include <stdio.h>

int main () {
    int sum = 0;
    int next;
    while (scanf("%d",&next) == 1) {
        sum = sum + next;
    }
    printf ("sum = %d\n",sum);
}

Overwriting sum.c


## Notes:

* ### If scanf reaches the end of the *stdin* input stream (i.e. end of file), *scanf* will return EOF (a negative number constant defined inside stdio.h) and the loop exits.

In [26]:
!gcc -o sum sum.c

In [27]:
!echo 10 20 30 | ./sum

sum = 60


In [28]:
!echo 5 10 15 20 > num.dat

In [29]:
!cat num.dat | ./sum

sum = 50


In [30]:
!echo 2000000000 4000000000 | ./sum

sum = 1705032704


# Exercise 1 : Write a program called *bigsum* that calculates a potentially large sum of big numbers read from *stdin* using variables of type *long long* instead of *int*.  

## Test your program using:

*!echo 2000000000 4000000000 | ./bigsum*

## It should output

*sum = 6000000000*

##Hint: Recall that *%lld* is the format specifier for *long long*.

## Here is a function that finds the smallest number in the input stream *stdin*.

In [31]:
%%writefile smallest.c
#include <stdio.h>
#include <limits.h>

int main () {
    int smallest = INT_MAX;
    int next;
    while (scanf("%d",&next) == 1) {
        if (next < smallest) {
            smallest = next;
        }
    }
    printf ("The smallest number is %d\n",smallest);
}

Overwriting smallest.c


In [32]:
!gcc -o smallest smallest.c

In [33]:
!echo -25 50 -75 100 -200 -3433 4500 | ./smallest

The smallest number is -3433


## Exercise 2 : Write a program called *smallest3* that finds the three smallest integers in *stdin*.

## Hint: Copy smallest.c to smallest3.c and modify it.

## Here is a program that finds the integer in *stdin* closest to a given number.

In [34]:
%%writefile closest.c
#include <stdio.h>
#include <stdlib.h>
#include <limits.h>

int main (int argc, char* argv[]) {
    if (argc < 2) {
        printf ("command usage: %s %s\n",argv[0],"center");
    }
    int center = atoi(argv[1]);
    int closest;
    int smallest_distance = INT_MAX;
    int next;
    while (scanf("%d",&next) == 1) {
        int distance = abs(next-center);
        if (distance < smallest_distance) {
            closest = next;
            smallest_distance = distance;
        }
    }
    printf ("The number closest to %d is %d\n",center,closest);
}

Overwriting closest.c


In [35]:
!gcc -o closest closest.c -lm

In [36]:
!echo 15 -7 25 -2 30 -5 29 37 -25 | ./closest 5

The number closest to 5 is -2


## Exercise 3 : Write a program called *closest3* that finds the three integers in *stdin* closest to a given command line argument *center*.

## Hint: Copy closest.c to closest3.c and modify it.